In [25]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import tensorflow as tf
import tensorflow_hub as hub
warnings.resetwarnings()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import os
import cv2
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import csv

train_dir = Path('input/training/')
test_dir = Path('input/validation/')
info = pd.read_csv("input/monkey_labels.txt", names=['Label','Latin Name', 'Common Name','Train Images', 'Validation Images'], skiprows=1)


ImportError: No module named ops

In [20]:
def read_data(filename):
    images = []
    labels = []
    paths = []
    i = 0
    for training_dir_path in glob.glob(filename + "/*"):
        label = info["Common Name"][i].rstrip()
        i=i+1
        for image_path in glob.glob(os.path.join(training_dir_path, "*.jpg")):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.resize(image, (1, 1))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
            labels.append(label)
            images.append(image)
            paths.append(image_path)
            
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    iencoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(iencoded)
    
    n = {'path': paths, 'labels':labels, 'int':integer_encoded}
    d = {'images':images,'labels':labels, 'int':integer_encoded, 'hot': onehot_encoded.tolist()}

    df = pd.DataFrame(n)
    return df

train = read_data("input/training")
train.to_csv('./input/train-paths-labels.csv', index=False) 



In [12]:
csv_path = tf.train.string_input_producer(['./input/train-paths-labels.csv'])
textReader = tf.TextLineReader()
_, csv_content = textReader.read(csv_path)
# _, rows = reader.read_up_to(file_queue, num_records=100*100)

record_defaults = [1], [""], [""]
        
columns = tf.decode_csv(csv_content, record_defaults=record_defaults)
        
labels = columns[0]
paths = columns[2]
# load images
content = tf.read_file(paths)
image = tf.image.decode_jpeg(content, channels=3)
image = tf.reshape(image, [10,10,3])


In [40]:
def read_labeled_image_list(image_list_file):
    df = pd.read_csv(image_list_file)
    return df["path"].tolist(), df["int"].tolist()

def read_images_from_disk(input_queue):
    label = input_queue[1]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    example.set_shape([50, 50, 3])
    return example, label

# Reads pathes of images together with their labels
image_list, label_list = read_labeled_image_list("./input/train-paths-labels.csv")

images = tf.convert_to_tensor(image_list, dtype=tf.string)
labels = tf.convert_to_tensor(label_list, dtype=tf.int32)

# Makes an input queue
input_queue = tf.train.slice_input_producer([images, labels], shuffle=True)

image, label = read_images_from_disk(input_queue)

# Optional Preprocessing or Data Augmentation
# tf.image implements most of the standard image augmentation
# image = preprocess_image(image)
# label = preprocess_label(label)

# Optional Image and Label Batching
image_batch, label_batch = tf.train.batch([image, label], batch_size=50)

print(image)
print(label)

Tensor("DecodeJpeg_12:0", shape=(50, 50, 3), dtype=uint8)
Tensor("input_producer_21/Gather_1:0", shape=(), dtype=int32)


In [ ]:
def generate_labelled_input_fn(csv_files, batch_size):
    def input_fn():
        file_queue = tf.train.string_input_producer(csv_files)
        reader = tf.TextLineReader(skip_header_lines=1)
#         _, rows = textReader.read(file_queue)
#         _, rows = reader.read(file_queue)
        _, rows = reader.read_up_to(file_queue, num_records=100*batch_size)
        expanded_rows = tf.expand_dims(rows, axis=-1)
     
        shuffled_rows = tf.train.shuffle_batch(
            [expanded_rows],
            batch_size=batch_size,
            capacity=20*batch_size,
            min_after_dequeue=5*batch_size,
            enqueue_many=True
        )
        
        record_defaults = [[0], [""], [""]]
        
        columns = tf.decode_csv(shuffled_rows, record_defaults=record_defaults)
        
        labels = columns[0]
        paths = columns[2]
#         paths = tf.concat(columns[2:], axis=0)
        
        
        # load images
        content = tf.read_file(paths)
        image = tf.image.decode_jpeg(content, channels=3)
#         image = tf.cast(image, tf.float32) / 255.
        image = tf.image.resize_images(image, [10, 10])
#         image = tf.reshape(image, [1,1,3])

        return {'images':  image}, labels
    
    return input_fn